In [2]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import json

In [3]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [4]:
users_reviews = []
for l in parse("review-Hawaii_10.json.gz"):
  users_reviews.append(l)

In [5]:
businesses = []
for l in parse("meta-Hawaii.json.gz"):
  businesses.append(l)

In [6]:
def MSE(predictions, labels):
    differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
    return sum(differences) / len(differences)

In [8]:
#user avg rating
user_ratings = {}

for review in users_reviews:
    user_id = review.get('user_id')
    rating = review.get('rating')

    if user_id is not None and rating is not None:
        if user_id not in user_ratings:
            user_ratings[user_id] = {'total_rating': 0, 'count': 0}
        
        user_ratings[user_id]['total_rating'] += rating
        user_ratings[user_id]['count'] += 1

user_avg_ratings = {user_id: data['total_rating'] / data['count'] 
                    for user_id, data in user_ratings.items() if data['count'] > 0}

for user_id in list(user_avg_ratings.keys())[:5]:
    print(f"User ID: {user_id}, Average Rating: {user_avg_ratings[user_id]:.2f}")

User ID: 113965417079576625433, Average Rating: 4.91
User ID: 116655819137293331166, Average Rating: 4.73
User ID: 100834119994550070853, Average Rating: 4.94
User ID: 103207214144482097315, Average Rating: 4.64
User ID: 108526171163172578599, Average Rating: 3.48


In [9]:
#businuess img count
image_counts = {}

for review in users_reviews:
    gmap_id = review.get('gmap_id')
    images = review.get('pics')

    if gmap_id and images:
        num_images = len(images)
        if gmap_id in image_counts:
            image_counts[gmap_id] += num_images
        else:
            image_counts[gmap_id] = num_images

In [10]:
response_counts = {}

for review in users_reviews:
    gmap_id = review.get('gmap_id')
    response = review.get('resp')

    if gmap_id and response:
        if gmap_id in response_counts:
            response_counts[gmap_id] += 1
        else:
            response_counts[gmap_id] = 1

In [11]:
users_data = [{'user_id': d['user_id'], 'gmap_id': d['gmap_id'], 'rating': d['rating'],'text': d['text']} 
              for d in users_reviews if 'user_id' in d and 'gmap_id' in d and 'rating' in d and 'text' in d]

business_dict = {d['gmap_id']: {'avg_rating': d.get('avg_rating', 0), 
                                'num_of_reviews': d.get('num_of_reviews', 0)}
                 for d in businesses if 'gmap_id' in d}

dataset = []
for user_review in users_data:
    gmap_id = user_review['gmap_id']
    if gmap_id in business_dict:
        merged_data = {**user_review, **business_dict[gmap_id]}
        dataset.append(merged_data)

In [12]:
#add features
for entry in dataset:
    gmap_id = entry.get('gmap_id')
    user_id = entry.get('user_id')
    entry['user_avg_rating'] = user_avg_ratings[user_id]
    entry['response_count'] = response_counts.get(gmap_id,0)
    entry['image_counts'] = image_counts.get(gmap_id, 0)
    

In [13]:
# split dataset
train_data = dataset[:int(len(dataset) * 0.9)]
test_data = dataset[int(len(dataset) * 0.9):]
train_users_reviews = users_reviews[:int(len(users_reviews) * 0.9)]
test_users_reviews = users_reviews[int(len(users_reviews) * 0.9):]

In [14]:
# lowercase, punctuation removed, no stemming
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
# stemmer = PorterStemmer()
for d in train_data:
    if d['text']:
        r = ''.join([c for c in d['text'].lower() if not c in punctuation])
        for w in r.split():
            # w = stemmer.stem(w)
            wordCount[w] += 1
len(wordCount)

311792

In [15]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort(reverse=True)
words = [x[1] for x in counts[:1000]]

In [16]:
wordID = dict(zip(words, range(len(words))))
wordSet = set(words)

In [17]:
def feature(datum):
    feat = [0]*len(words)
    if d['text']:
        r = ''.join([c for c in d['text'].lower() if not c in punctuation])
        for w in r.split():
            if w in words:
                feat[wordID[w]] += 1
    feat.append(1) #offset
    return feat

In [18]:
train_X = [feature(d) for d in train_data]
train_y = [u['rating'] for u in train_users_reviews]
test_X = [feature(d) for d in test_data]
test_y = [u['rating'] for u in test_users_reviews]

In [ ]:
# regularized regression
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(train_X, train_y)
train_predictions = clf.predict(train_X)
test_predictions = clf.predict(test_X)

TypeError: 'module' object is not callable

In [ ]:
train_mse = MSE(train_predictions, train_y)
test_mse = MSE(test_predictions, test_y)
print(f'train mse: {train_mse}')
print(f'test mse: {test_mse}')